In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from citipy import citipy
from config import weather_api_key

In [2]:
# Create 2,000 lats and lngs to be paired as coordinates
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)

In [3]:
# Turn the zip into a list of coordinates
coordinates = list(lats_lngs)

In [4]:
# Create a list of cities by looping through the coordinates and using citipy to determine closest city
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)
# Check the number of cities produced by the coordinate list
len(cities)

745

In [5]:
# Define base url
baseUrl = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Perform API call on cities to determine lat/lng, max temp, % humidity, % cloudiness, wind speed, sky description

# Create iteration that will call the APIs for the city weather data
#Create empty list for city data and to loop through
city_data = []
# Print the beginning of logging
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loops through all the cities in the list
for i, city in enumerate(cities):
    # Group cities in sets of 50 to log process as we find the weather data for each city
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count = 1
    # Create endpoint url with each city
    city_url = baseUrl + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        city_weather = requests.get(city_url).json()
        # Parse out needed information: lat/lng, max temp, % humidity, % cloudiness, wind speed, sky description
        city_lat = city_weather["coord"]["lat"]
        city_lon = city_weather["coord"]["lon"]
        city_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_sky = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Append city data with parsed out informatino
        city_data.append({
            "City" : city.title(),
            "Country" : city_country,
            "Lat" : city_lat,
            "Lng" : city_lon,
            "Max Temp" : city_temp,
            "Humidity" : city_humidity,
            "Cloudiness" : city_clouds,
            "Wind Speed" : city_wind,
            "Current Description" : city_sky
        })
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

ecord 42 of Set 6 | vardo
Processing Record 43 of Set 6 | karratha
Processing Record 44 of Set 6 | eregli
Processing Record 45 of Set 6 | riyadh
Processing Record 46 of Set 6 | burgeo
Processing Record 47 of Set 6 | shuangcheng
Processing Record 48 of Set 6 | ushtobe
Processing Record 49 of Set 6 | buckingham
Processing Record 50 of Set 6 | nanortalik
Processing Record 1 of Set 7 | corinto
Processing Record 2 of Set 7 | kavieng
Processing Record 3 of Set 7 | buncrana
Processing Record 4 of Set 7 | sorland
Processing Record 5 of Set 7 | derzhavinsk
Processing Record 6 of Set 7 | mujiayingzi
Processing Record 7 of Set 7 | chadiza
Processing Record 8 of Set 7 | point pleasant
Processing Record 9 of Set 7 | tiksi
Processing Record 10 of Set 7 | pacific grove
Processing Record 11 of Set 7 | sungaipenuh
Processing Record 12 of Set 7 | gat
Processing Record 13 of Set 7 | maningrida
Processing Record 14 of Set 7 | jyvaskyla
Processing Record 15 of Set 7 | saleaula
City not found. Skipping...
P

In [8]:
# Convert to dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df

City Country      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0           Dikson      RU  73.5069   80.5464    -34.28        80           0   
1     Kruisfontein      ZA -34.0033   24.7314     60.01        90           0   
2            Vaini      TO -21.2000 -175.2000     86.00        79          75   
3     Punta Arenas      CL -53.1500  -70.9167     51.80        62           0   
4    Mount Gambier      AU -37.8333  140.7667     64.40        59          75   
..             ...     ...      ...       ...       ...       ...         ...   
682      Bundaberg      AU -24.8500  152.3500     87.01        51           0   
683        Paamiut      GL  61.9940  -49.6678     23.92        89           0   
684    Meadow Lake      US  34.8014 -106.5436     44.01        36           1   
685        Demirci      TR  39.0461   28.6589     33.84        94          70   
686       Kamloops      CA  50.6665 -120.3192     35.60        80          90   

     Wind Speed Current Description  
0          6.85           clear sky  
1          3.51           clear sky  
2         13.80       broken clouds  
3         26.46           clear sky  
4          8.05       broken clouds  
..          ...                 ...  
682       14.97           clear sky  
683       15.23           clear sky  
684       11.50           clear sky  
685        1.99       broken clouds  
686       10.36     overcast clouds  

[687 rows x 9 columns]

In [12]:
# Create output file
output_data_file = "WeatherPy_Database.csv"
# Export the city_data_df as CSV
city_data_df.to_csv(output_data_file, index_label= "City ID")